<h2 align='center'>Codebasics ML Course: ML Flow Tutorial</h2>


In [16]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings

warnings.filterwarnings("ignore")


In [17]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(
    n_samples=1000,
    n_features=10,
    n_informative=2,
    n_redundant=8,
    weights=[0.9, 0.1],
    flip_y=0,
    random_state=42,
)

np.unique(y, return_counts=True)


(array([0, 1]), array([900, 100], dtype=int64))

In [18]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)


#### Handle class imbalance


In [19]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(X_train_res, return_counts=True)


(array([-5.51207083, -5.14120195, -4.69421946, ...,  5.84871724,
         5.91086505,  6.02415826]),
 array([1, 1, 1, ..., 1, 1, 1], dtype=int64))

### Track Experiments


In [20]:
models = [
    (
        "Logistic Regression",
        {"C": 1, "solver": "liblinear"},
        LogisticRegression(),
        (X_train, y_train),
        (X_test, y_test),
    ),
    (
        "Random Forest",
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(),
        (X_train, y_train),
        (X_test, y_test),
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": "logloss"},
        XGBClassifier(),
        (X_train, y_train),
        (X_test, y_test),
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": "logloss"},
        XGBClassifier(),
        (X_train_res, y_train_res),
        (X_test, y_test),
    ),
]


In [21]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]

    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)


In [22]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost


In [23]:
# Initialize MLflow
mlflow.set_experiment("Anomaly Detection V1")
mlflow.set_tracking_uri("http://localhost:5000")

# for i, element in enumerate(models):
#     model_name = element[0]
#     params = element[1]
#     model = element[2]
#     report = reports[i]

#     with mlflow.start_run(run_name=model_name):
#         mlflow.log_params(params)
#         mlflow.log_metrics(
#             {
#                 "accuracy": report["accuracy"],
#                 "recall_class_1": report["1"]["recall"],
#                 "recall_class_0": report["0"]["recall"],
#                 "f1_score_macro": report["macro avg"]["f1-score"],
#             }
#         )

#         if "XGB" in model_name:
#             mlflow.xgboost.log_model(model, "model")
#         else:
#             mlflow.sklearn.log_model(model, "model")


### Register the Model


In [24]:
model_name = "XGB-Smote"
run_id = "3ad3473128104944b04619c6aa2ee69e"
model_uri = f"runs:/{run_id}/model"

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)


Registered model 'XGB-Smote' already exists. Creating a new version of this model...
2025/03/29 15:02:57 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 6


🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/841932292576404006/runs/3ad3473128104944b04619c6aa2ee69e
🧪 View experiment at: http://localhost:5000/#/experiments/841932292576404006


Created version '6' of model 'XGB-Smote'.


### Load the Model


In [25]:
# load directly
import mlflow
import mlflow.xgboost
import pandas as pd

# model_uri = "runs:/3ad3473128104944b04619c6aa2ee69e/model"

# Load model explicitly
loaded_model = mlflow.xgboost.load_model(model_uri)

# Ensure input_data is correctly formatted
input_data = pd.DataFrame(X_test)  # Convert to DataFrame

# Make predictions
y_pred = loaded_model.predict(input_data)
print("First 4 Predictions:", y_pred[:4])


First 4 Predictions: [0 0 0 0]


In [26]:
# load with version
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]


array([0, 0, 0, 0])

In [ ]:
# load with alias
model_uri = f"models:/{model_name}@challanger"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]


array([0, 0, 0, 0])

### Transition the Model to Production


In [29]:
current_model_uri = f"models:/{model_name}@challanger"
production_model_name = "anomaly-detection-prod"

client = mlflow.MlflowClient()
client.copy_model_version(
    src_model_uri=current_model_uri, dst_name=production_model_name
)


Successfully registered model 'anomaly-detection-prod'.
Copied version '2' of model 'XGB-Smote' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1743240888827, current_stage='None', description='', last_updated_timestamp=1743240888827, name='anomaly-detection-prod', run_id='3ad3473128104944b04619c6aa2ee69e', run_link='', source='models:/XGB-Smote/2', status='READY', status_message=None, tags={}, user_id='', version='1'>

## Loading Production Model


In [ ]:
model_version = 1
prod_model_uri = f"models:/{production_model_name}@champion"

loaded_model = mlflow.xgboost.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]


array([0, 0, 0, 0])

Please refer to following to learn more about model registry

https://mlflow.org/docs/latest/model-registry.html#model-registry-workflows to learn
